In [ ]:
# | default_exp frontmatter

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

import jupyter_black
import nbdev.showdoc as showdoc
from fastcore.test import *

jupyter_black.load()

In [ ]:
# | export
from typing import Any
from textwrap import dedent
from frontmatter.util import u  # type: ignore[import-untyped]
from frontmatter.default_handlers import YAMLHandler  # type: ignore[import-untyped]

## Frontmatter

Later on, we will use `frontmatter` to make our code generator more powerful. Let's build some helpers

In [ ]:
# | hide
# | export
class FrontMatter:
    def __init__(self, h: YAMLHandler | None = None):  # type: ignore[no-any-unimported]
        if h is None:
            h = YAMLHandler()
        self.handler = h

    def split(self, raw_content: str, *, encoding: str = "utf-8") -> Any:
        raw_content = u(raw_content, encoding).strip()

        try:
            fm, content = self.handler.split(raw_content)
        except ValueError:
            return None, raw_content

        return fm, content

    def parse(self, raw_frontmatter: Any, *, metadata: Any = None) -> Any:
        if metadata is None:
            metadata = {}

        try:
            raw_frontmatter = self.handler.load(raw_frontmatter)
        except Exception as e:
            msg = dedent(
                f"""
            ===
            There is an error with the following yaml (front matter)
            
            ```
            {raw_frontmatter}
            ```

            ===

            """
            )

            print(msg)
            raise e

        if isinstance(raw_frontmatter, dict):
            metadata.update(raw_frontmatter)

        return metadata

    def get_content(self, template: str) -> str:
        frontmatter, content = self.split(template)
        ret: str = content.strip()
        return ret

    def get_frontmatter_source(self, template: str) -> str:
        resp = self.split(template)
        frontmatter, content = resp
        if frontmatter:
            ret: str = frontmatter.strip()
            return ret
        return ""

In [ ]:
model = """
---
reference:  "sigla-{{ node.attrs.name | lower }}-model"
---
class {{ name }}Model(models.Model): # {{ reference }}
    {% for child in children -%}
    {{ child | render }}
    {% endfor %}
"""

fm = FrontMatter().get_frontmatter_source(model)
test_eq(fm, 'reference:  "sigla-{{ node.attrs.name | lower }}-model"')

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()